In [1]:
import wbgapi as wb   # I use wb as a namespace in all my work
import pandas as pd   # optional but highly recommended for wbgapi
import matplotlib.pyplot as plt
import csv, sqlite3

%matplotlib inline

In [2]:
#read country_list dats to get country codes of interested countries
countries=pd.read_csv("//Users/enginsan/Downloads/data-verbose.csv",
names=["Country", "Country_Code", "Country_Name"])

In [3]:
countries.head()

,Country,Country_Code,Country_Name
0,COUNTRY,AFG,Afghanistan
1,COUNTRY,ALB,Albania
2,COUNTRY,DZA,Algeria
3,COUNTRY,AND,Andorra
4,COUNTRY,AGO,Angola


In [4]:
#drop column Country:
countries.drop(columns=['Country'])

,Country_Code,Country_Name
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,AND,Andorra
4,AGO,Angola
...,...,...
245,SSD,South Sudan
246,SDF,Sudan (former)
247,CHI,Channel islands
248,IMN,Isle of Man


I am interested in OECD countries
The OECD’s members are: Austria, Australia, Belgium, Canada, 
Chile, Colombia, Costa Rica, Czech Republic, Denmark, Estonia, 
Finland, France, Germany, Greece, Hungary, Iceland, Ireland, Israel, 
Italy, Japan, Korea, Latvia, Lithuania, Luxembourg, Mexico, 
the Netherlands, New Zealand, Norway, Poland, Portugal, 
Slovak Republic, Slovenia, Spain, Sweden, 
Switzerland, Turkey, the United Kingdom and the United States

In [5]:
oecd_countries=['Austria', 'Australia', 'Belgium', 'Canada', 
'Chile', 'Colombia', 'Costa Rica', 'Czechia', 'Denmark', 'Estonia', 
'Finland', 'France', 'Germany', 'Greece', 
'Hungary', 'Iceland', 'Ireland', 'Israel', 
'Italy', 'Japan', 'Korea', 'Latvia', 'Lithuania', 
'Luxembourg', 'Mexico', 
'Netherlands (Kingdom of the)', 'New Zealand', 'Norway', 'Poland', 'Portugal', 
'Slovakia', 'Slovenia', 'Spain', 'Sweden', 
'Switzerland', 'Turkiye', 'United Kingdom of Great Britain and Northern Ireland', 
'United States of America']

In [6]:
countries_oecd=countries['Country_Name'].isin(oecd_countries)
countries_selected =countries[countries_oecd]

In [7]:
countries_selected.head()

,Country,Country_Code,Country_Name
8,COUNTRY,AUS,Australia
9,COUNTRY,AUT,Austria
16,COUNTRY,BEL,Belgium
30,COUNTRY,CAN,Canada
34,COUNTRY,CHL,Chile


In [8]:
countries_selected_codelist=countries_selected['Country_Code'].values.tolist()
countries_selected_codelist

['AUS',
 'AUT',
 'BEL',
 'CAN',
 'CHL',
 'COL',
 'CRI',
 'CZE',
 'DNK',
 'EST',
 'FIN',
 'FRA',
 'DEU',
 'GRC',
 'HUN',
 'ISL',
 'IRL',
 'ISR',
 'ITA',
 'JPN',
 'LVA',
 'LTU',
 'LUX',
 'MEX',
 'NLD',
 'NZL',
 'NOR',
 'POL',
 'PRT',
 'SVK',
 'SVN',
 'ESP',
 'SWE',
 'CHE',
 'TUR',
 'GBR',
 'USA']

In [9]:
#check if these countries in WDI database
wb.economy.info(countries_selected_codelist)

ConnectionError: HTTPSConnectionPool(host='api.worldbank.org', port=443): Max retries exceeded with url: /v2/en/sources/2/concepts?per_page=1000&page=1&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1184f1e50>: Failed to establish a new connection: [Errno 12] Cannot allocate memory'))

Selected indicators:

In [10]:
indicator_dict={'gdp_percapita':'NY.GDP.PCAP.KD',
'population':'SP.POP.TOTL',
'fdi_received':'BX.KLT.DINV.WD.GD.ZS',
'hightechexports':'TX.VAL.TECH.MF.ZS',
'unemployment' :'SL.UEM.TOTL.ZS',
'current_account':'BN.CAB.XOKA.GD.ZS',
'lifeexpectancy':'SP.DYN.LE00.IN',
'researchanddev': 'GB.XPD.RSDV.GD.ZS',
'investment': 'NE.GDI.FTOT.ZS',
'consumerprices':'FP.CPI.TOTL.ZG',
'merchandise_trade':'TG.VAL.TOTL.GD.ZS',
'laborforce_female':'SL.TLF.TOTL.FE.ZS',
'domesticcreditto_privatesector':'FS.AST.PRVT.GD.ZS',
'gini_index':'SI.POV.GINI',
'governmentexpenditure_education':'SE.XPD.TOTL.GB.ZS',
'militaryexpenditure':'MS.MIL.XPND.GD.ZS',
'scientificjournal':'IP.JRN.ARTC.SC',
'internetusers':'IT.NET.USER.ZS'}

In [11]:
indicator_dict.keys()
indicator_dict.values()

dict_values(['NY.GDP.PCAP.KD', 'SP.POP.TOTL', 'BX.KLT.DINV.WD.GD.ZS', 'TX.VAL.TECH.MF.ZS', 'SL.UEM.TOTL.ZS', 'BN.CAB.XOKA.GD.ZS', 'SP.DYN.LE00.IN', 'GB.XPD.RSDV.GD.ZS', 'NE.GDI.FTOT.ZS', 'FP.CPI.TOTL.ZG', 'TG.VAL.TOTL.GD.ZS', 'SL.TLF.TOTL.FE.ZS', 'FS.AST.PRVT.GD.ZS', 'SI.POV.GINI', 'SE.XPD.TOTL.GB.ZS', 'MS.MIL.XPND.GD.ZS', 'IP.JRN.ARTC.SC', 'IT.NET.USER.ZS'])

In [12]:
indic_df= pd.DataFrame.from_dict(indicator_dict, orient='index')
indic_df

,0
gdp_percapita,NY.GDP.PCAP.KD
population,SP.POP.TOTL
fdi_received,BX.KLT.DINV.WD.GD.ZS
hightechexports,TX.VAL.TECH.MF.ZS
unemployment,SL.UEM.TOTL.ZS
current_account,BN.CAB.XOKA.GD.ZS
lifeexpectancy,SP.DYN.LE00.IN
researchanddev,GB.XPD.RSDV.GD.ZS
investment,NE.GDI.FTOT.ZS
consumerprices,FP.CPI.TOTL.ZG


In [13]:
indic_df.reset_index()
#indic_df = indic_df.index.to_frame().join(indic_df)

,index,0
0,gdp_percapita,NY.GDP.PCAP.KD
1,population,SP.POP.TOTL
2,fdi_received,BX.KLT.DINV.WD.GD.ZS
3,hightechexports,TX.VAL.TECH.MF.ZS
4,unemployment,SL.UEM.TOTL.ZS
5,current_account,BN.CAB.XOKA.GD.ZS
6,lifeexpectancy,SP.DYN.LE00.IN
7,researchanddev,GB.XPD.RSDV.GD.ZS
8,investment,NE.GDI.FTOT.ZS
9,consumerprices,FP.CPI.TOTL.ZG


In [14]:
indic_df.columns=['value_1']

In [15]:
indic_df.sort_values(by=['value_1'], inplace=True)

In [16]:

indic_df.reset_index()

,index,value_1
0,current_account,BN.CAB.XOKA.GD.ZS
1,fdi_received,BX.KLT.DINV.WD.GD.ZS
2,consumerprices,FP.CPI.TOTL.ZG
3,domesticcreditto_privatesector,FS.AST.PRVT.GD.ZS
4,researchanddev,GB.XPD.RSDV.GD.ZS
5,scientificjournal,IP.JRN.ARTC.SC
6,internetusers,IT.NET.USER.ZS
7,militaryexpenditure,MS.MIL.XPND.GD.ZS
8,investment,NE.GDI.FTOT.ZS
9,gdp_percapita,NY.GDP.PCAP.KD


In [17]:
indicator_list=list(indic_df['value_1'])
indicator_list

['BN.CAB.XOKA.GD.ZS',
 'BX.KLT.DINV.WD.GD.ZS',
 'FP.CPI.TOTL.ZG',
 'FS.AST.PRVT.GD.ZS',
 'GB.XPD.RSDV.GD.ZS',
 'IP.JRN.ARTC.SC',
 'IT.NET.USER.ZS',
 'MS.MIL.XPND.GD.ZS',
 'NE.GDI.FTOT.ZS',
 'NY.GDP.PCAP.KD',
 'SE.XPD.TOTL.GB.ZS',
 'SI.POV.GINI',
 'SL.TLF.TOTL.FE.ZS',
 'SL.UEM.TOTL.ZS',
 'SP.DYN.LE00.IN',
 'SP.POP.TOTL',
 'TG.VAL.TOTL.GD.ZS',
 'TX.VAL.TECH.MF.ZS']

In [18]:
headers=list(indic_df.index)
headers

['current_account',
 'fdi_received',
 'consumerprices',
 'domesticcreditto_privatesector',
 'researchanddev',
 'scientificjournal',
 'internetusers',
 'militaryexpenditure',
 'investment',
 'gdp_percapita',
 'governmentexpenditure_education',
 'gini_index',
 'laborforce_female',
 'unemployment',
 'lifeexpectancy',
 'population',
 'merchandise_trade',
 'hightechexports']

Now it is time to exctract data using indicator and country code list,
time range is chosen as 2012-2022

In [19]:
wdi_data = wb.data.DataFrame(indicator_list, countries_selected_codelist, 
time=range(2012, 2022), index=['time', 'economy'], labels=True)

In [20]:
wdi_data.head(20)

,,Time,Country,BN.CAB.XOKA.GD.ZS,BX.KLT.DINV.WD.GD.ZS,FP.CPI.TOTL.ZG,FS.AST.PRVT.GD.ZS,GB.XPD.RSDV.GD.ZS,IP.JRN.ARTC.SC,IT.NET.USER.ZS,MS.MIL.XPND.GD.ZS,NE.GDI.FTOT.ZS,NY.GDP.PCAP.KD,SE.XPD.TOTL.GB.ZS,SI.POV.GINI,SL.TLF.TOTL.FE.ZS,SL.UEM.TOTL.ZS,SP.DYN.LE00.IN,SP.POP.TOTL,TG.VAL.TOTL.GD.ZS,TX.VAL.TECH.MF.ZS
time,economy,,,,,,,,,,,,,,,,,,,,
YR2021,USA,2021,United States,-3.566160,2.114876,4.697859,216.308017,3.45705,NaN,91.753208,3.457977,21.186197,61829.845627,NaN,39.8,46.149185,5.350,76.329268,332031554.0,20.114080,19.893869
YR2020,USA,2020,United States,-2.835383,0.656980,1.233584,215.778145,3.46777,455855.57,90.620470,3.696010,21.316016,58451.606715,12.654216,39.7,46.148933,8.050,76.980488,331511512.0,18.194591,19.483849
YR2019,USA,2019,United States,-2.066080,1.477875,1.812210,191.241255,3.17049,438020.45,89.430285,3.434568,20.978794,60698.011299,15.517134,41.5,46.148783,3.670,78.787805,328329953.0,19.693236,18.672652
YR2018,USA,2018,United States,-2.142160,1.045704,2.442583,180.461848,3.01010,435033.88,88.498903,3.323866,20.918142,59607.393660,15.780731,41.4,46.051084,3.900,78.639024,326838199.0,20.835684,18.474054
YR2017,USA,2017,United States,-1.887394,1.955211,2.130110,191.630055,2.90432,430198.17,87.274889,3.320541,20.648162,58207.578310,15.900826,41.2,46.058367,4.360,78.539024,325122128.0,20.304362,19.258818
YR2016,USA,2016,United States,-2.119372,2.537498,1.261583,183.644222,2.85350,428476.45,85.544421,3.422588,20.372829,57292.538783,15.973817,41.1,45.944943,4.870,78.539024,323071755.0,19.797502,22.411343
YR2015,USA,2015,United States,-2.243472,2.809148,0.118627,180.732080,2.78700,428203.78,74.554202,3.481428,20.449164,56762.729452,16.052464,41.2,45.985847,5.280,78.690244,320738994.0,20.970387,21.380941
YR2014,USA,2014,United States,-2.108055,1.435021,1.622223,185.399004,2.71786,428140.59,73.000000,3.690961,20.299954,55675.386854,16.004614,41.5,46.108991,6.170,78.841463,318386329.0,22.979617,20.467144
YR2013,USA,2013,United States,-2.015384,1.710668,1.464833,184.572858,2.70215,426596.66,71.400000,4.032661,19.722427,54830.784066,16.024628,40.7,46.084722,7.370,78.741463,316059947.0,23.206131,20.178305


index values and column names to be adjusted as follows:

In [21]:
wdi_data=wdi_data.droplevel(level=0)

In [22]:
list1=list(wdi_data.columns[0:2])
list1

['Time', 'Country']

In [23]:
list1.extend(headers)

In [24]:
wdi_data.columns=list1

In [25]:
wdi_data.head()

,Time,Country,current_account,fdi_received,consumerprices,domesticcreditto_privatesector,researchanddev,scientificjournal,internetusers,militaryexpenditure,investment,gdp_percapita,governmentexpenditure_education,gini_index,laborforce_female,unemployment,lifeexpectancy,population,merchandise_trade,hightechexports
economy,,,,,,,,,,,,,,,,,,,,
USA,2021,United States,-3.566160,2.114876,4.697859,216.308017,3.45705,NaN,91.753208,3.457977,21.186197,61829.845627,NaN,39.8,46.149185,5.35,76.329268,332031554.0,20.114080,19.893869
USA,2020,United States,-2.835383,0.656980,1.233584,215.778145,3.46777,455855.57,90.620470,3.696010,21.316016,58451.606715,12.654216,39.7,46.148933,8.05,76.980488,331511512.0,18.194591,19.483849
USA,2019,United States,-2.066080,1.477875,1.812210,191.241255,3.17049,438020.45,89.430285,3.434568,20.978794,60698.011299,15.517134,41.5,46.148783,3.67,78.787805,328329953.0,19.693236,18.672652
USA,2018,United States,-2.142160,1.045704,2.442583,180.461848,3.01010,435033.88,88.498903,3.323866,20.918142,59607.393660,15.780731,41.4,46.051084,3.90,78.639024,326838199.0,20.835684,18.474054
USA,2017,United States,-1.887394,1.955211,2.130110,191.630055,2.90432,430198.17,87.274889,3.320541,20.648162,58207.578310,15.900826,41.2,46.058367,4.36,78.539024,325122128.0,20.304362,19.258818


In [26]:
#reshape DataFrame from wide format to long format
wdi_data1 = pd.melt(wdi_data, id_vars=['Time', 'Country'], 
value_vars=headers)


In [27]:
wdi_data1.head()

,Time,Country,variable,value
0,2021,United States,current_account,-3.566160
1,2020,United States,current_account,-2.835383
2,2019,United States,current_account,-2.066080
3,2018,United States,current_account,-2.142160
4,2017,United States,current_account,-1.887394


USING SQLLITE TO STORE DATA AND USE IN DASHBOARD (TO BE CREATED IN R)

In [28]:
#!pip install jupysql
%load_ext sql

In [29]:
con = sqlite3.connect("dash_data1.db")
cur = con.cursor()
print(con)


In [30]:
# Drop the table if already exists.
cur.execute("DROP TABLE IF EXISTS WDI")

#write wdi data to WDI table:

In [31]:

wdi_data1.to_sql("WDI", con, if_exists='replace', index=False,method="multi")

6660

In [32]:
%sql sqlite:///dash_data1.db 

Connecting to 'sqlite:///dash_data1.db'

In [33]:
#check with a query
%sql select * from WDI

Running query in 'sqlite:///dash_data1.db'

Time,Country,variable,value
2021,United States,current_account,-3.56615967017701
2020,United States,current_account,-2.83538257957979
2019,United States,current_account,-2.06607966606092
2018,United States,current_account,-2.14216028970484
2017,United States,current_account,-1.88739358215214
2016,United States,current_account,-2.11937229657855
2015,United States,current_account,-2.2434721228246
2014,United States,current_account,-2.10805505161045
2013,United States,current_account,-2.01538414033942
2012,United States,current_account,-2.57236812074952


In [35]:
#Test a query
%sql select Time, Country as 'Country', variable as 'Population' from WDI \
where Country='Germany' and Time = 2020 and variable = 'population'

Running query in 'sqlite:///dash_data1.db'

Time,Country,Population
2020,Germany,population
